In [2]:
#All neccesary classes for project

#general
import seaborn as sns
import scipy.stats as ss
import pandas as pd
import numpy as np
import os
import pickle

#for preprocessing
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import tensorflow as tf

#for machine learning
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import expon, reciprocal
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz

#for evaluation
from sklearn.metrics import confusion_matrix

from sklearn.metrics import f1_score

#for utility packages
from Utilities.utilities import import_data
from Utilities.utilities import DataFrameSelector
from Utilities.utilities import CategoricalEncoder
from Utilities.utilities import display_scores
from Utilities.utilities import pipeline_transform
from Utilities.utilities import reset_graph
from Utilities.models import DNN_Model
from Utilities.models import cross_val_score_dnn
from functools import partial

#image manipulation
from PIL import Image as PI
from resizeimage import resizeimage
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from keras.utils.data_utils import get_file
import vgg_preprocessing
import tensorflow.contrib.slim as slim
from tensorflow.contrib.slim.nets import resnet_v2
import pnasnet as nas

#Import Custom Functions
from Utilities.model_builder import get_image
from Utilities.model_builder import get_file_lists
from Utilities.model_builder import parse_record
from Utilities.model_builder import get_batch
from Utilities.model_builder import build_iterator
from Utilities.model_builder import get_values
from Utilities.models import log_dir_build
from Utilities.utilities import generate_image
from Utilities.utilities import generate_image_array
from Utilities.cell_net_predictor import Binary_Categorical_Predictor

Using TensorFlow backend.


# Load the Data

Here we will load the training and validation data in order to do training

In [3]:
#Import TFRecords for both Training and Testing of the Dat
#Use the build_image_data.py to create these sets from your data
#train_list, val_list = get_file_lists("C:/tmp/model_training_data/Dog_Cat/tf_record")
labels = ["Cat", "Dog"]


#Nasnet Model Location
nas_net_model = 'C:/AI/pnas_net/model.ckpt'
animal_net_logs = 'C:/AI/Animal_Net_v2/logs'
animal_net_model = 'C:/AI/Animal_Net_v2/model/cell_net'

test_image = generate_image_array('C:/AI/Animal_Net/test_image/Cat.jpg', 299, 299)

# Build the Neural Network on top of Large Nas-Net

Here we will build the Nas-Net and then stack our own network on top

In [4]:
#Reset the graph 
reset_graph()

#Set constants for Neural Network
dropout_rate = 0.5
learning_rate = 0.01
n_hidden1 = 500
n_hidden2 = 250
n_hidden3 = 100
n_hidden4 = 50
n_hidden5 = 25
n_hidden6 = 10
n_final_layer = 2


#Placeholder for input data
X = tf.placeholder(tf.float32, shape=[None, 299, 299, 3], name="input")
y = tf.placeholder(tf.int32, shape=[None], name="output")
training = tf.placeholder_with_default(False, shape=(), name = 'training')

#Define initalizer and batch normalization layers
bn_batch_norm_layer = partial(tf.layers.batch_normalization, training=training, momentum=0.9)
he_init = tf.contrib.layers.variance_scaling_initializer()


#Import the Nas_Net and build it
with slim.arg_scope(nas.pnasnet_large_arg_scope()):
    net, end_points = nas.build_pnasnet_large(X, num_classes=1001, is_training=False)
    
    #A saver to load the pretrained Data
    saver = tf.train.Saver()
    
    #For getting predictions from Original Network
    soft_max_pna = tf.get_default_graph().get_tensor_by_name("final_layer/predictions:0")
    
    #Load in the noder where we are going to connect our own network
    last_feature_node = tf.get_default_graph().get_tensor_by_name("final_layer/dropout/Identity:0")

    

with tf.name_scope("DNN_Classifier"):
    #Use a stop layer to freeze all the layers beneath in Nas-Net
    with tf.name_scope("Hidden_Layer_1"):
        stop_layer = tf.stop_gradient(last_feature_node)
        hidden1 = tf.layers.dense(stop_layer, n_hidden1, name="hidden1", kernel_initializer=he_init)
        hidden1_drop = tf.layers.dropout(hidden1, dropout_rate, training=training)
        bn1 = bn_batch_norm_layer(hidden1_drop)
        bn1_act = tf.nn.elu(bn1)
    
    with tf.name_scope("Hidden_Layer_2"):
        hidden2 = tf.layers.dense(bn1_act, n_hidden2, name="hidden2", kernel_initializer=he_init)
        hidden2_drop = tf.layers.dropout(hidden2, dropout_rate, training=training)
        bn2 = bn_batch_norm_layer(hidden2_drop)
        bn2_act = tf.nn.elu(bn2)
    
    with tf.name_scope("Hidden_Layer_3"):
        hidden3 = tf.layers.dense(bn2_act, n_hidden3, name="hidden3", kernel_initializer=he_init)
        hidden3_drop = tf.layers.dropout(hidden3, dropout_rate, training=training)
        bn3 = bn_batch_norm_layer(hidden3_drop)
        bn3_act = tf.nn.elu(bn3)
    
    with tf.name_scope("Hidden_Layer_4"):
        hidden4 = tf.layers.dense(bn3_act, n_hidden4, name="hidden4", kernel_initializer=he_init)
        hidden4_drop = tf.layers.dropout(hidden4, dropout_rate, training=training)
        bn4 = bn_batch_norm_layer(hidden4_drop)
        bn4_act = tf.nn.elu(bn4)
    
    with tf.name_scope("Hidden_Layer_5"):
        hidden5 = tf.layers.dense(bn4_act, n_hidden5, name="hidden5", kernel_initializer=he_init)
        hidden5_drop = tf.layers.dropout(hidden5, dropout_rate, training=training)
        bn5 = bn_batch_norm_layer(hidden5_drop)
        bn5_act = tf.nn.elu(bn5)
    
    with tf.name_scope("Hidden_Layer_6"):
        hidden6 = tf.layers.dense(bn5_act, n_hidden6, name="hidden6", kernel_initializer=he_init)
        hidden6_drop = tf.layers.dropout(hidden6, dropout_rate, training=training)
        bn6 = bn_batch_norm_layer(hidden6_drop)
        bn6_act = tf.nn.elu(bn6)
    
    with tf.name_scope("Final_Layer"):
        logits_before_bn = tf.layers.dense(bn6_act, n_final_layer, name="outputs")
        logits = bn_batch_norm_layer(logits_before_bn, name="logits")
        softmax = tf.nn.softmax(logits)
    
    with tf.name_scope("loss"):
            xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
            loss = tf.reduce_mean(xentropy, name="loss")
            loss_summary = tf.summary.scalar('loss_summary', loss)
            
    with tf.name_scope("train"):
        global_step = tf.Variable(0, trainable=False, name='global_step')
        optimizer = tf.train.AdamOptimizer(learning_rate)
        training_op = optimizer.minimize(loss, global_step=global_step)
        

    with tf.name_scope("eval"):
        correct = tf.nn.in_top_k(logits, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
        accuracy_summary = tf.summary.scalar('accuracy_summary', accuracy)
        
        
            
#Variables for global initialization
saver2 = tf.train.Saver()
init = tf.global_variables_initializer()
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

INFO:tensorflow:A GPU is available on the machine, consider using NCHW data format for increased speed on GPU.


In [15]:
with tf.Session() as sess:
    #Initalizer all variables
    init.run()
    
    #Restore the pretrained variables from Nas-Net
    saver.restore(sess, nas_net_model)
    
    
    #Save all of these variables to the new Cell_Net Model
    saver2.save(sess, cell_net_model)

INFO:tensorflow:Restoring parameters from C:/tmp/pnas_net/model.ckpt


In [5]:
#For Exporting Graph
filewriter = tf.summary.FileWriter(cell_net_logs, tf.get_default_graph())

# Build Testing

Here we will make sure the pre-trained network loaded and is effective with Imagenet.
If correct using first of Imagenet test it should print 286

In [16]:
with tf.Session() as sess:
    saver2.restore(sess, cell_net_model)
    y_raw = soft_max_pna.eval(feed_dict={X: test_image})
    
frame = pd.DataFrame(y_raw)
item = frame.iloc[0,:]
item = pd.Series.idxmax(item)
item

INFO:tensorflow:Restoring parameters from C:/tmp/Cell_Net/model_test/cell_net


286

# Train the Network

Here we are going to train the network. Accuracy/Loss is recorded

In [10]:
#For doing the initial training
#Total number of epochs to train
num_epochs = 50
steps_between_test_save = 20
batch_size = 5

# Save the Graph into a file with Filewriter and add summaries for this session
model_path = log_dir_build(cell_net_logs, "cell_out")
filewriter = tf.summary.FileWriter(model_path, tf.get_default_graph())

with tf.Session() as sess:
    #restore saver, build iterator, set the step to the global step
    saver2.restore(sess, cell_net_model)
    
    #Build iterators to pull train and test data
    iterator_train = build_iterator(True, train_list, batch_size, num_epochs=num_epochs, num_parallel_calls=8)
    iterator_test = build_iterator(False, val_list, batch_size, num_epochs=num_epochs, num_parallel_calls=4)
    next_train = iterator_train.get_next()
    next_test = iterator_test.get_next()
    
    #Set up the global steps
    step = 0
    print("Loaded model. Training network initially. Logs into: " + model_path)

    #Iterate through training 
    while step < num_epochs:
        #Get training data
        X_val, y_val = next_train
        X_val, y_val = get_values(sess, X_val, y_val)
        
        #run Training Op
        sess.run([training_op, extra_update_ops, accuracy_summary], feed_dict={X: X_val, y: y_val, training: True})
        
        #Maybe Test Accuracy
        if (step % steps_between_test_save) == 0:
            X_val, y_val = next_test
            X_val, y_val = get_values(sess, X_val, y_val)
            acc_sum, loss_sum, loss_val = sess.run([accuracy_summary, loss_summary, loss], feed_dict = {X: X_val, y: y_val, training: False})
            filewriter.add_summary(acc_sum, step)
            filewriter.add_summary(loss_sum, step)
            print("Step: " + str(step) + " Loss: " + str(loss_val))
            saver2.save(sess, cell_net_model)
        step = step + 1
            
    #Finish the final Model
    saver2.save(sess, cell_net_model)
    print("Done!")

INFO:tensorflow:Restoring parameters from C:/tmp/Cell_Net/model/cell_net
Loaded model. Training network initially. Logs into: C:/tmp/Cell_Net/logs/cell_out-run-20180531192055/
Step: 0 Loss: 0.681009
Step: 20 Loss: 0.448954
Step: 40 Loss: 0.339628
Done!


In [11]:
#For picking up with additional training
#Total number of epochs to train
num_epochs = 1000
steps_between_test_save = 20
batch_size = 5

with tf.Session() as sess:
    #restore saver, build iterator, set the step to the global step
    saver2.restore(sess, cell_net_model)
    
    #Build iterators to pull train and test data
    iterator_train = build_iterator(True, train_list, batch_size, num_epochs=num_epochs, num_parallel_calls=8)
    iterator_test = build_iterator(False, val_list, batch_size, num_epochs=num_epochs, num_parallel_calls=4)
    next_train = iterator_train.get_next()
    next_test = iterator_test.get_next()
    
    #Set up the global steps
    step = tf.train.global_step(sess, global_step)
    print("Loaded model. Starting training from step: " + str(step) + ", Logging in directory: " + model_path)

    #Iterate through training 
    while step < num_epochs:
        #Get training data
        X_val, y_val = next_train
        X_val, y_val = get_values(sess, X_val, y_val)
        
        #run Training Op
        sess.run([training_op, extra_update_ops, accuracy_summary], feed_dict={X: X_val, y: y_val, training: True})
        
        #Maybe Test Accuracy
        if (step % steps_between_test_save) == 0:
            X_val, y_val = next_test
            X_val, y_val = get_values(sess, X_val, y_val)
            acc_sum, loss_sum, loss_val = sess.run([accuracy_summary, loss_summary, loss], feed_dict = {X: X_val, y: y_val, training: False})
            filewriter.add_summary(acc_sum, step)
            filewriter.add_summary(loss_sum, step)
            print("Step: " + str(step) + " Loss: " + str(loss_val))
            saver2.save(sess, cell_net_model)
        step = step + 1
            
    #Finish the final Model
    saver2.save(sess, cell_net_model)
    print("Done!")

INFO:tensorflow:Restoring parameters from C:/tmp/Cell_Net/model/cell_net
Loaded model. Starting training from step: 50, Logging in directory: C:/tmp/Cell_Net/logs/cell_out-run-20180531192055/
Step: 60 Loss: 0.415838
Step: 80 Loss: 0.357053
Step: 100 Loss: 0.517353
Step: 120 Loss: 0.135984
Step: 140 Loss: 0.284187
Step: 160 Loss: 0.0725838
Step: 180 Loss: 0.202192
Step: 200 Loss: 0.0777485
Step: 220 Loss: 0.0684065
Step: 240 Loss: 0.173998
Step: 260 Loss: 0.0411993
Step: 280 Loss: 0.244819
Step: 300 Loss: 0.189933
Step: 320 Loss: 0.0505728
Step: 340 Loss: 0.0339866
Step: 360 Loss: 0.204192
Step: 380 Loss: 0.21926
Step: 400 Loss: 0.0374186
Step: 420 Loss: 0.0241346
Step: 440 Loss: 0.125693
Step: 460 Loss: 0.0724637
Step: 480 Loss: 0.0583581
Step: 500 Loss: 0.0268569
Step: 520 Loss: 0.0746905
Step: 540 Loss: 0.313732
Step: 560 Loss: 0.0576057
Step: 580 Loss: 0.0195325
Step: 600 Loss: 0.254252
Step: 620 Loss: 0.17301
Step: 640 Loss: 0.0225835
Step: 660 Loss: 0.0231231
Step: 680 Loss: 0.217

# Test the Model

Use this to run a sample through the network and get a softmax estimate for each class


In [12]:
cat = generate_image_array('C:/tmp/Cell_Net/test_image/Cat.jpg', 299, 299)
dog = generate_image_array('C:/tmp/Cell_Net/test_image/Dog.jpg', 299, 299)

In [ ]:
with tf.Session() as sess:
    #restore graph from meta and restore variables
    new_saver = tf.train.import_meta_graph(cell_net_model + '.meta')
    new_saver.restore(sess, cell_net_model)
    soft = tf.get_default_graph().get_tensor_by_name("DNN_Classifier/Softmax:0")
    steps = tf.get_default_graph().get_tensor_by_name("DNN_Classifier/train/global_step:0")
    current_step = steps.eval()

In [6]:
with tf.Session() as sess:
    #restore graph from meta and restore variables
    new_saver = tf.train.import_meta_graph(cell_net_model + '.meta')
    new_saver.restore(sess, cell_net_model)
    soft = tf.get_default_graph().get_tensor_by_name("DNN_Classifier/Softmax:0")
    input_tensor = tf.get_default_graph().get_tensor_by_name("input:0")
    val = soft.eval(feed_dict={input_tensor: cat})

INFO:tensorflow:Restoring parameters from C:/tmp/Cell_Net/model/cell_net


array([[ 0.97931594,  0.02068399]], dtype=float32)

In [18]:
predictor = Binary_Categorical_Predictor(cell_net_model, labels)

In [19]:
predictor.print_image_probability('C:/tmp/Cell_Net/test_image/Jimmy.jpg')

INFO:tensorflow:Restoring parameters from C:/tmp/Cell_Net/model/cell_net
Probability of Cat: 0.0262442
Probability of Dog: 0.973756
